[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davHub/edges2car/blob/master/cGAN.ipynb)

# ***pix2pix implementation***

## Get data from drive

In [1]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_id = '18JHHTKNrxY40YdC5g70SBuBrmDwg1hVL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('cars_color.tar.gz')

!tar -xf cars_color.tar.gz

!mkdir processed
!mkdir resized


In [0]:
file_id = '1neEd0g57M3IGDrIQCMGc7dY4Ezm7NpJS'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_color_edge.tar.gz')

!tar -xf test_color_edge.tar.gz

#!mkdir processed
#!mkdir resized


In [5]:
file_id = '1qjIlr0jcmvj86pg_YG1Qg2wX86OMPGJW'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('cars_new.tar')

!tar -xf cars_new.tar

!mkdir processed
!mkdir resized

In [0]:
!tar czvf data_car.tar.gz resized


In [0]:
drive_folder_id = '1MqdArcCnHnEQTp3gXEEPNqlm5aFWFIzf'
name = "model-23"
#name = "6.jpg"

def get_file_from_drive(folder_id, file_name):
  file_list = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  for file in file_list:
    if file['title'] == file_name:
      return file['id']

def upload_file_to_drive(file_name, file_data):
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentString(file_data)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))
  
def upload_data_system(name):
  try:
    downloaded = drive.CreateFile({'id': get_file_from_drive(drive_folder_id, name)})
    downloaded.GetContentFile(name)
  except Exception as err:
      print(name, 'not found')
      pass

def save_in_drive(name,folder_id = '1u6UBZpQuEz9xpqu2Fjre4jxj2XpYi843'):
  file = drive.CreateFile({'parents':[{u'id': folder_id}]})
  file.SetContentFile(name)
  file.Upload()
    
upload_data_system(name+'.data-00000-of-00001')
upload_data_system(name+'.index')
upload_data_system(name+'.meta')
#upload_data_system('vgg_16.ckpt')
#save_in_drive('data_car.tar.gz', folder_id=drive_folder_id)

#save_in_drive('checkpoint', folder_id=drive_folder_id)
#save_in_drive(name+'.data-00000-of-00001', folder_id=drive_folder_id)
#save_in_drive(name+'.index', folder_id=drive_folder_id)
#save_in_drive(name+'.meta', folder_id=drive_folder_id)


In [54]:
!rm processed/*
!rm resized/*
!rm tcar/*


In [99]:
!ls
!ls -l cars_new |wc -l
!ls -l cars_color |wc -l
!ls -l processed |wc -l
!ls -l resized |wc -l
!ls -l test |wc -l

#!mkdir processed
#!mkdir resized
#!pip install Image
#!ls edges2car

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pickle
import numpy as np
import tensorflow as tf
import json
import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as draw
import cv2
import time
from PIL import Image, ImageFilter, ImageOps, ImageEnhance
from IPython.display import HTML, display
import os
import scipy.misc
from glob import glob

import pickle

from tensorflow.contrib.slim.nets import vgg
slim = tf.contrib.slim



## Parameters

In [0]:
# data parameters
update_data = True

h=256
w=256

# Learning parameters
EPOCHS = 50
batch_size = 1
test_batch_size = 5
learning_rate_d = 0.0002
learning_rate_g = 0.0002
dropout = 0.5
beta1 = 0.5

df_dim = 64
gf_dim = 64

new_network = True
train = True
ratio_train = 0.95

#k_initializer = tf.random_normal_initializer(0, 0.02)
k_initializer = tf.contrib.layers.xavier_initializer()

k_regulizer = tf.contrib.layers.l2_regularizer(scale=0.1)

EPS = 1e-12

In [0]:
def display_img(img):
  im = np.uint8(img)
  plt.imshow(im)
  plt.axis('off')
  plt.show()

## Preprocessing data and Data Augmentation

In [23]:
step = 30
def Norme(p1,p2,p3,p4):
    n = math.sqrt((p1[0]-p3[0])*(p1[0]-p3[0]) + (p2[0]-p4[0])*(p2[0]-p4[0]))    
    return n

def get_contours(img, lib=None,seuil=50):
  w,h,c = img.shape
  imgC = None
  if lib is 'CV2' :
    imgC = cv2.Canny(img,w,h)
    #imgC = cv2.bitwise_not(imgC)
  elif lib is 'PIL':
    imgC = Image.fromarray(img.astype('uint8'), 'RGB')
    imgC = imgC.filter(ImageFilter.CONTOUR)
  else :
    imgP = Image.fromarray(img.astype('uint8'), 'RGB')
    imgC = Image.new(imgP.mode,imgP.size)
    for i in range(1,ligne-1):
      for j in range(1,colonne-1):
        p1 = imgP.getpixel((j-1,i))
        p2 = imgP.getpixel((j,i-1))
        p3 = imgP.getpixel((j+1,i))
        p4 = imgP.getpixel((j,i+1))
        n = Norme(p1,p2,p3,p4)
        if n < seuil:
          p = (255,255,255)
        else:
          p = (0,0,0)
        imgC.putpixel((j-1,i-1),p)

  return np.asarray(imgC)
  
def get_car_color(img, l=5):
  taux = 25
  w,h,c = img.shape
  start_w = int(w/2)
  start_h = int(h/2)
  
  #imr = img[start_w:start_w+l,start_h:start_h+l,0] // taux
  #print(imr)
  #cr = np.reshape(imr,(l*l,))
  #r = np.bincount(cr).argmax() * taux
  #cg = np.reshape(img[start_w:start_w+l,start_h:start_h+l,1] // taux,(l*l,))
  #g = np.bincount(cg).argmax() * taux
  #cb = np.reshape(img[start_w:start_w+l,start_h:start_h+l,2] // taux,(l*l,))
  #b = np.bincount(cb).argmax() * taux
  
  #color = [r,g,b]
  
  color = np.mean(np.mean(img[start_w:start_w+l,start_h:start_h+l,:],axis=0),axis=0)
  
  return color
  
def is_white(color, seuil=230):
  return color[0]>seuil and color[1]>seuil and color[2]>seuil

def is_bg_white(img, l=8):
  w,h,c = img.shape
  c = 0
  window = []
  window.append(img[0:l,0:l,:])
  window.append(img[0:l,h-(2*l):h-(2*l)+l,:])
  window.append(img[w-(2*l):w-(2*l)+l,0:l,:])
  window.append(img[w-(2*l):w-(2*l)+l,h-(2*l):h-(2*l)+l,:])
  for i in window:
    color = np.mean(np.mean(i,axis=0),axis=0)
    if is_white(color, seuil=230):
      c+=1
  
  return c >2

def progress(value, max=100):
  return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
  """.format(value=value, max=max))
  
print('processing data')
out = display(progress(0, 100), display_id=True)
prog = 0
count = 0
countError=0
NewFile =  glob("cars_new/*")
ColorFile =  glob("cars_color/*")
ImageFile = NewFile + ColorFile
length = len(ImageFile)
for idx, file in enumerate(ImageFile):
    try:
      img = Image.open(file)
      img = img.resize((w, h),Image.ANTIALIAS)
      colonne,ligne = img.size
      img = np.asarray(img)
      
      if is_bg_white(img,l=10):
        count +=1

        edges = get_contours(img, lib='CV2', seuil=50)
        color = get_car_color(img, l=2)

        dim = edges.shape
        #print(dim)
        process_img = np.zeros((dim[0],dim[1],3),dtype='uint8')

        #print(process_img.shape)

        #process_img[:,:,0] = (1-edges/255)*color[0] 
        #process_img[:,:,1] = (1-edges/255)*color[1] 
        #process_img[:,:,2] = (1-edges/255)*color[2] 

        process_img[:,:,0] = np.where(edges/255 == 1, color[0],255) 
        process_img[:,:,1] = np.where(edges/255 == 1, color[1],255)
        process_img[:,:,2] = np.where(edges/255 == 1, color[2],255) 


        #display_img(edges)
        #display_img(process_img)
        #display_img(img)

        if idx%step ==0:
          display_img(img[:,:,:3])
          #display_img(edges[:,:,:3])
          display_img(process_img[:,:,:3])

        #print(type(process_img[0,0,0]),process_img[:,:,:3].shape)
        #print(type(img[0,0,0]),img[:,:,:3].shape)

        scipy.misc.imsave("processed/"+file[9:], process_img[:,:,:3])
        scipy.misc.imsave("resized/"+file[9:], img[:,:,:3])
    except (IOError,ValueError,KeyError):#,ValueError,KeyError, TypeError):
      countError+= 1
      #os.remove(file)
    if prog % 17 ==0:
      percent =  (prog+1) / length * 100
      out.update(progress(percent, 100))
    prog += 1
    

data_dict = glob("processed/*")
random.shuffle(data_dict)    
train_data = data_dict[:int(ratio_train*len(data_dict))]
test_data =  data_dict[int(ratio_train*len(data_dict)):]
print("Nb image : ",len(data_dict))
print(len(train_data))
print(len(test_data))
    


In [0]:

countError = 0
ImageFile =  glob("resized/*")
for idx, file in enumerate(ImageFile):
    try:
      img0 = Image.open(file)
      img = np.asarray(img0)
      color = get_color(img)
      
      edges = plt.imread("nb1/"+file[8:])
      
      lim = 200
      
      e = edges.copy()
      
      posB = np.where(np.mean(edges, axis=2)<lim)
      e[posB] = [255,255,255]
      posC = np.where(np.mean(edges, axis=2)>lim)
      e[posC]=color
      
      if idx%70 ==0:
        display_img(img)
        display_img(edges)
        display_img(e)
    except (IOError,ValueError):
       countError+= 1#print("ERROR")
    img0.close()

    
print("Error " + str(countError))



In [11]:
!ls
name = '85.jpg'
img = Image.open('cars_new/'+name)
img = np.asarray(img)
plt.imshow(img)
plt.axis('off')
plt.show()

In [9]:

data_dict = glob("processed/*")
random.shuffle(data_dict)    
train_data = data_dict[:int(ratio_train*len(data_dict))]
test_data =  data_dict[int(ratio_train*len(data_dict)):]
print("Nb image : ",len(data_dict))
print(len(train_data))
print(len(test_data))

In [0]:
def preprocess(img):
  # [0,1] => [-1,1]
  return np.float32(img)/255 * 2 - 1

def deprocess(img):
  # [-1, 1] => [0, 1]
  return np.uint8(255.*((img+1.)/2))


In [0]:
def get_image(file):
  img = plt.imread(file)
  out = preprocess(img[:,:,:3])
  return out

def next_batch_2(batch_size, index):
  batch = [ get_image("processed/"+batch_file[10:]) for batch_file in train_data[index*batch_size:(index+1)*batch_size]]
  label = [ get_image("resized/"+batch_file[10:]) for batch_file in train_data[index*batch_size:(index+1)*batch_size]]
  return np.array(batch).astype(np.float32), np.array(label).astype(np.float32)

def next_batch(batch_size, index, training=True):
  inputs = []
  targets = []
  #print("Batch from : " + str(cb))
  
  t = train_data if training else test_data
  batch_files = t[index*batch_size:(index+1)*batch_size]
  if training :
    for name in batch_files:
        file = name[10:]
        inp = plt.imread("processed/"+file)
        
        inp = preprocess(inp[:,:,:3])
        #print(type(inp[0,0,0]),inp.shape)
        inputs.append(inp)

        tar = plt.imread("resized/"+file)
        tar = preprocess(tar[:,:,:3])
        targets.append(tar)
        
    return inputs, targets
  
  else :
    for name in batch_files:
      file = name[5:]
      img = plt.imread("tcar/"+file)
      #print(img)
      inp = preprocess(img[:,:,:3])
      inputs.append(inp)
    return inputs


In [0]:
global_step = 0

## pix2pix Architecture

In [0]:
def discriminator(inputs, targets, reuse=False, training=True):
  with tf.variable_scope("discriminator", reuse=reuse):
    # image is 256 x 256 x (input_c_dim + output_c_dim)
   
    input = tf.concat([inputs, targets], axis=3)
    print(input)
    l = tf.layers.conv2d(input, df_dim, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)

    l = tf.layers.conv2d(l, df_dim*2, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)
    l = tf.layers.batch_normalization(l, training=True) 

    l = tf.layers.conv2d(l, df_dim*4, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)
    l = tf.layers.batch_normalization(l, training=True)

    l = tf.layers.conv2d(l, df_dim*8, 4, strides=(1,1), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)
    l = tf.layers.batch_normalization(l, training=True) 

    l = tf.layers.conv2d(l, df_dim*8, 3, strides=(1,1), activation=tf.nn.leaky_relu, padding='valid', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)
    l = tf.layers.conv2d(l, 1, 3, strides=(1,1), padding='valid', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
    print(l)
    

    return tf.nn.sigmoid(l), l


In [0]:
def encoder(inputs):
  # encoding
  print(inputs)
  e1 = tf.layers.conv2d(inputs, gf_dim, 4, strides=(2,2), activation=None, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e1) 
  e2 = tf.layers.conv2d(e1, gf_dim*2, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e2)
  e2 = tf.layers.batch_normalization(e2, training=True) 
  
  e3 = tf.layers.conv2d(e2, gf_dim*4, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e3)
  e3 = tf.layers.batch_normalization(e3, training=True) 
 
  e4 = tf.layers.conv2d(e3, gf_dim*8, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e4)
  e4 = tf.layers.batch_normalization(e4, training=True) 
 
  e5 = tf.layers.conv2d(e4, gf_dim*8, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e5)
  e5 = tf.layers.batch_normalization(e5, training=True) 
  e6 = tf.layers.conv2d(e5, gf_dim*8, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e6)
  e6 = tf.layers.batch_normalization(e6, training=True) 
 
  e7 = tf.layers.conv2d(e6, gf_dim*8, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e7)
  e7 = tf.layers.batch_normalization(e7, training=True) 
 
  e8 = tf.layers.conv2d(e7, gf_dim*8, 4, strides=(2,2), activation=tf.nn.leaky_relu, padding='same', kernel_initializer=k_initializer, kernel_regularizer= k_regulizer)
  print(e8)
  e8 = tf.layers.batch_normalization(e8, training=True) 
  
  return e1,e2,e3,e4,e5,e6,e7,e8
 


In [0]:
def decoder(inputs):
  
  e1,e2,e3,e4,e5,e6,e7,e8 = inputs
  
  d8=tf.layers.conv2d_transpose(e8, gf_dim*8, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d8)
  d8 = tf.layers.batch_normalization(d8, training=True) 
  d8 = tf.nn.dropout(d8, keep_prob=1 - dropout)
  
  i = tf.concat([d8,e7], axis=3)
  d7=tf.layers.conv2d_transpose(i, gf_dim*8, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d7)
  d7 = tf.layers.batch_normalization(d7, training=True) 
  d7 = tf.nn.dropout(d7, keep_prob=1 - dropout)
  
  i = tf.concat([d7,e6], axis=3)
  d6=tf.layers.conv2d_transpose(i, gf_dim*8, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d6)
  d6 = tf.layers.batch_normalization(d6, training=True) 
  d6 = tf.nn.dropout(d6, keep_prob=1 - dropout)
  
  i = tf.concat([d6,e5], axis=3)
  d5=tf.layers.conv2d_transpose(i, gf_dim*8, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d5)
  d5 = tf.layers.batch_normalization(d5, training=True) 
  
  i = tf.concat([d5,e4], axis=3)
  d4=tf.layers.conv2d_transpose(i, gf_dim*4, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d4)
  d4 = tf.layers.batch_normalization(d4, training=True) 
  
  i = tf.concat([d4,e3], axis=3)
  d3=tf.layers.conv2d_transpose(i, gf_dim*2, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d3)
  d3 = tf.layers.batch_normalization(d3, training=True) 
  
  i = tf.concat([d3,e2], axis=3)
  d2=tf.layers.conv2d_transpose(i, gf_dim, kernel_size=4, strides=(2,2), activation=tf.nn.relu, padding="same", kernel_initializer=k_initializer)
  print(d2)
  d2 = tf.layers.batch_normalization(d2, training=True) 
  
  i = tf.concat([d2,e1], axis=3)
  d1=tf.layers.conv2d_transpose(i, 3, kernel_size=4, strides=(2,2), activation=None, padding="same", kernel_initializer=k_initializer)
  print(d1)
  
  return tf.tanh(d1)


In [0]:
def generator(inputs):
  with tf.variable_scope("generator") as scope:
    enc = encoder(inputs)
    dec = decoder(enc)
    return dec


In [0]:
def model(inputs, targets):
  
  lambda_gen = 1.
  lambda_L1 = 100.
  
  G = generator(inputs)
  
  D_real, D_real_logits = discriminator(targets,inputs, reuse=False)
  D_fake, D_fake_logits = discriminator(G, inputs, reuse=True)
  
  #D loss
  D_loss =  tf.reduce_mean(-(tf.log(D_real + EPS) + tf.log(1 - D_fake + EPS)))
  
  # G loss
  G_loss_GAN = tf.reduce_mean(-tf.log(D_fake + EPS))
  G_loss_L1 = tf.reduce_mean(tf.abs(targets - G))
  
  G_loss = lambda_gen * G_loss_GAN + lambda_L1 * G_loss_L1
  
  
  t_vars = tf.trainable_variables()

  D_vars = [var for var in t_vars if 'discriminator' in var.name]
  G_vars = [var for var in t_vars if 'generator' in var.name]
  
  D_optim = tf.train.AdamOptimizer(learning_rate_d, beta1=beta1) \
                          .minimize(D_loss, var_list=D_vars)
  G_optim = tf.train.AdamOptimizer(learning_rate_g, beta1=beta1) \
                          .minimize(G_loss, var_list=G_vars)

  return D_loss, D_optim, G_loss, G_optim, G

In [0]:
def debug(epoch=None,step=None,loss=None, accuracy=None, time=None, name=""):
  debug=""
  if epoch is not None :
    debug+="Epoch : [{}] - ".format(epoch)
  
  if step is not None :
    debug+="Step : [{}] - ".format(step)
  
  if time is not None :
    debug+="Time : %4.2f min\n" % (time/60000)
  if loss is not None :
    debug += ""
    debug += name +" - "
  
    debug+="Loss : {} - ".format(loss)
  
  if accuracy is not None :
    debug+="Accuracy : {}".format(accuracy)
    
  return debug 


## Training

In [92]:
tf.reset_default_graph()
print("Init Tensors")
graph = tf.Graph()
with graph.as_default():
  inputs = tf.placeholder( tf.float32, shape=[batch_size]+[256, 256, 3] )
  targets = tf.placeholder( tf.float32 , shape=[batch_size]+[256, 256, 3])
  
  discrim_loss, discrim_optim, gen_loss, gen_optim, gen_predict = model(inputs,targets)
  
  saver = tf.train.Saver(max_to_keep=6)

  
print("Start Training")
with tf.Session(graph=graph) as sess:
  if False:
    sess.run(tf.global_variables_initializer())
    global_step = 0
  else:
    saver.restore(sess, tf.train.latest_checkpoint('./'))
    #saver.restore(sess, 'model-10')

  num_batches = int(len(train_data)/batch_size)
  print(num_batches)
  start_time = time.time()
  for epoch in range(EPOCHS):
    random.shuffle(train_data)
    for step in range(num_batches):
  
      train_inputs, train_targets = next_batch(batch_size, step)
      #print(type(train_inputs),len(train_inputs))
      #print(type(train_targets),train_targets)
     
      _ , D_loss = sess.run([discrim_optim, discrim_loss], feed_dict={
          inputs: train_inputs,
          
          targets: train_targets
      })
      
      _ , G_loss = sess.run([gen_optim, gen_loss], feed_dict={
          inputs: train_inputs,
          targets: train_targets
      })
      
      _ , G_loss = sess.run([gen_optim, gen_loss], feed_dict={
          inputs: train_inputs,
          targets: train_targets
      })
      
      if step%(num_batches//4)==0:
        time_now = time.time()
        debug_D = debug(epoch=epoch, time=time_now - start_time,step = step, loss=D_loss,name="Dicriminator")
        print(debug_D)
        debug_G = debug(loss=G_loss,name="Generator")
        print(debug_G)

      if step% (num_batches//3) == 0 :
        G_predict = sess.run([gen_predict], feed_dict={
          inputs: train_inputs
        })
        
        #display_img(deprocess(train_targets[0]))
        display_img(deprocess(G_predict[0][0]))

    if epoch%3 == 0 :
      print("Saving model")
      saver.save(sess, "./model", global_step=global_step)

    global_step = global_step + 1


## Testing

In [17]:
ImageFile =  glob("test_color_edge/*")
print(ImageFile)
count_test=0
for idx, file in enumerate(ImageFile):
    try:
      img = Image.open(file)
      img = img.resize((w, h),Image.ANTIALIAS)
      img = np.asarray(img)
      if len(img.shape)<3 or img.shape[0] is not w or img.shape[1] is not h :
        continue
      if idx%8 ==0:
        display_img(img)

      
      scipy.misc.imsave("tcar/"+file[16:], img)
      count_test +=1
    except (IOError,ValueError,KeyError):
      countError+= 1
      
print(count_test)

In [16]:
!ls
!mkdir tcar
#!rm tcar/*
!ls test_color_edge
#!rm -r test_color_edge

In [22]:
test_batch_size =1

test_data = glob("tcar/*6*")
print(len(test_data))

show =5

tf.reset_default_graph()
print("Init Tensors")
graph = tf.Graph()
with graph.as_default():
  inputs = tf.placeholder( tf.float32, shape=[test_batch_size]+[h, w, 3] )
  targets = tf.placeholder( tf.float32 , shape=[test_batch_size]+[h, w, 3])
  
  discrim_loss, discrim_optim, gen_loss, gen_optim, gen_predict = model(inputs,targets)
  
  saver = tf.train.Saver(max_to_keep=4)

  
print("Start Training")
with tf.Session(graph=graph) as sess:
  #saver.restore(sess, tf.train.latest_checkpoint('./'))
  saver.restore(sess, 'model-23')

  num_batches = int(len(test_data)/test_batch_size)
  print(num_batches)
  start_time = time.time()
  random.shuffle(test_data)
  for step in range(num_batches):
    test_inputs = next_batch(test_batch_size, step,training=False)
    G_predict = sess.run(gen_predict, feed_dict={
        inputs: test_inputs
    })
    display_img(deprocess(test_inputs[0]))
    display_img(deprocess(G_predict[0]))